In [ ]:
# TODO
# Logic to build job template in AAP

In [3]:
import requests
import json
import os
from datetime import datetime

aap_credential = os.getenv("aap_credential")

In [9]:
AAP_URL = "https://example-aap-aap.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com"
USERNAME = "admin"
PASSWORD = aap_credential

TEMPLATE_NAME = f"Remediation Job Template {datetime.now().strftime('%Y%m%d_%H%M%S')}"
PROJECT_ID = 10              # ID of the project in AAP
INVENTORY_ID = 2            # ID of the inventory in AAP
PLAYBOOK = "remediation.yaml"      # Playbook file in your project
CREDENTIAL_ID = 4          # Credential ID for SSH/machine or vault

In [5]:
# Disable SSL warnings for self-signed certs (not for production)
requests.packages.urllib3.disable_warnings()

In [6]:
# --- Authenticate ---
session = requests.Session()
session.auth = (USERNAME, PASSWORD)
session.verify = False  # Change to True if using valid SSL certs
session.headers.update({'Content-Type': 'application/json'})

In [7]:
# --- Create Job Template ---
url = f"{AAP_URL}/api/controller/v2/job_templates/"

payload = {
    "name": TEMPLATE_NAME,
    "job_type": "run",
    "inventory": INVENTORY_ID,
    "project": PROJECT_ID,
    "playbook": PLAYBOOK,
    "execution_environment": 2,
    "ask_verbosity_on_launch": False,
    "ask_variables_on_launch": True,
    "become_enabled": True
}

response = session.post(url, json=payload)
response_data = response.json()
job_template_id = response_data.get("id")

if response.status_code == 201:
    print(f"✅ Job template '{TEMPLATE_NAME}' created successfully.")
else:
    print(f"❌ Failed to create job template: {response.status_code}")

✅ Job template 'Remediation Job Template 20250605_031117' created successfully.


In [12]:
# Write job_template_id to a json file for next stage to use
with open("job_template_id.json", "w") as f:
    json.dump({"job_template_id": job_template_id}, f)

In [10]:
# Add credential to job template

url = f"{AAP_URL}/api/controller/v2/job_templates/{job_template_id}/credentials/"
payload = {
    "id": CREDENTIAL_ID
}

response = session.post(url, json=payload)
print(response.status_code)

if response.status_code == 204:
    print(f"✅ Credential added to Job template successfully.")
else:
    print(f"❌ Failed to add credential to job template: {response.status_code}")

204
✅ Credential added to Job template successfully.
